### Лабораторна робота №9
#### Виконав: Студент групи КН-24-1 Соломка Борис Олегович

## Алгоритми на рядках

## Вступ

**Мета роботи:** освоїти низку основних алгоритмів на рядках засобами мови Python.

**Завдання:**
- Реалізувати алгоритм пошуку підрядка в рядку
- Реалізувати z-функцію і застосовувати її в алгоритмах аналізу рядків

## Хід роботи

### 1. Наївний пошук шаблонів

Наївний пошук шаблонів є найпростішим методом серед алгоритмів пошуку шаблонів. Він перевіряє, чи всі символи основного рядка збігаються з шаблоном. Цей алгоритм корисний для невеликих текстів.

**Постановка задачі:** Дано текст 𝑡 і зразок 𝑝 (вважаємо, що |𝑝| < |𝑡|). Завдання: знайти всі входження зразка 𝑝 у текст 𝑡.

In [1]:
def naive_match(p, t):
    assert len(p) <= len(t) # припускаємо, що довжина шаблону не більша за довжину тексту
    occurrences = []
    for i in range(0, len(t)-len(p)+1): # для кожного вирівнювання шаблону на текст
        match = True # припускаємо, що є збіг, доки не доведено зворотнє
        for j in range(0, len(p)): # для кожної позиції шаблону
            if t[i+j] != p[j]:
                match = False # принаймні один символ не збігається
                break
        if match:
            occurrences.append(i)
    return occurrences

In [2]:
# Перевіримо роботу алгоритму
naive_match('needle', 'needleneedleneedle')

[0, 6, 12]

### 2. Z-функція

**Постановка задачі:** Нехай дано рядок 𝑠 довжини 𝑛. Тоді Z-функція від цього рядка – це масив довжини 𝑛, 𝑖-ий елемент якого дорівнює найбільшому числу символів, починаючи з позиції 𝑖, які співпадають із першими символами рядка 𝑠.

In [3]:
def z_func(s):
    # Ініціалізуємо масив Z, в якому будемо зберігати значення Z-функції
    Z = [len(s)] + [0] * (len(s) - 1) # Встановлюємо довжину рядка s у першому елементі масиву Z, а решту заповнюємо нулями
    
    if len(s) <= 1:
        return Z
    
    # Початкова порівняння s[1:] з префіксом
    for i in range(1, len(s)):
        # Порівнюємо символи s[i] та s[i-1]. Якщо вони однакові, збільшуємо значення Z[1]
        if s[i] == s[0]:
            Z[1] += 1
        else:
            break # Якщо знайдений символ не співпадає з попереднім, припиняємо процес
    
    # Встановлюємо значення змінних r і l
    r, l = 0, 0
    if Z[1] > 0:
        r, l = Z[1], 1
    
    # Обчислення Z-функції для кожного k від 2 до довжини рядка s
    for k in range(2, len(s)):
        if k > r:
            # Випадок 1: k за межами поточного діапазону [l, r]
            for i in range(k, len(s)):
                # Порівнюємо символи s[i-k] та s[i]. Якщо вони співпадають, збільшуємо значення Z[k]
                if s[i-k] == s[i]:
                    Z[k] += 1
                else:
                    break
            # Оновлюємо значення змінних r та l
            if Z[k] > 0:
                r, l = k + Z[k] - 1, k
        else:
            # Випадок 2: k знаходиться всередині поточного діапазону [l, r]
            # Обчислюємо довжину бета (nbeta) та значення Z-функції на попередньому кроці (Zkp)
            nbeta = r - k + 1
            Zkp = Z[k - l]
            if nbeta > Zkp:
                # Підвипадок 2а: Zkp виграє
                Z[k] = Zkp
            else:
                # Підвипадок 2б: Порівнюємо символи, що знаходяться безпосередньо після r
                nmatch = 0
                for i in range(r+1, len(s)):
                    if s[i-k] == s[i]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                Z[k] = r - k + 1
    
    return Z

In [4]:
# Перевіримо роботу Z-функції
z_func('abracadabra')

[11, 0, 0, 1, 0, 1, 0, 4, 0, 0, 1]

### 3. Пошук підрядка в рядку за допомогою Z-функції

**Постановка задачі:** Нехай 𝑡 – текст, 𝑝 – зразок. Необхідно знайти всі входження зразка 𝑝 у текст 𝑡.

In [5]:
def zMatch(p, t):
    # Формуємо розширений рядок s, де p – це шаблон, а t – текст, який шукаємо.
    s = p + "$" + t
    # Обчислюємо масив Z для рядка s.
    Z = z_func(s)
    # Ініціалізуємо список occurrences для зберігання позицій, де відбувається збіг з шаблоном.
    occurrences = []
    # Проходимо по всім позиціям збігу шаблону з текстом у рядку s.
    # Починаємо з позиції, що відповідає кінцю шаблону (len(p) + 1) і до кінця рядка s.
    for i in range(len(p) + 1, len(s)):
        # Якщо значення Z на поточній позиції рівне довжині шаблону, то це означає, що відбувся збіг.
        if Z[i] == len(p):
            # Додаємо позицію, де відбувся збіг, до списку occurrences.
            # Позиція розраховується як i – (len(p) + 1), оскільки ми розглядаємо розширений рядок s,
            # де шаблон p розташований на початку, перед "$".
            occurrences.append(i - (len(p) + 1))
    # Повертаємо список знайдених збігів.
    return occurrences

In [6]:
# Перевіримо роботу алгоритму пошуку підрядка
t, p = "абабагаламага", "аб"
calculated_z = z_func(p + "$" + t)
print(calculated_z)
print(zMatch(p, t))

[16, 0, 0, 2, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
[0, 2, 4]


### 4. Стиснення рядка за допомогою Z-функції

**Постановка задачі:** Дано рядок 𝑠 довжини 𝑛. Необхідно знайти найкоротше його «стисле» представлення, тобто знайти такий рядок 𝑡 найменшої довжини, що 𝑠 можна представити у вигляді конкатенації однієї або декількох копій 𝑡.

In [7]:
def compress_with_z(s):
    # Обчислюємо Z-вектор для рядка s
    z_vec = z_func(s)
    # Проходимо по елементам Z-вектора, починаючи з індексу 1
    for i in range(1, len(s)):
        # Перевіряємо, чи виконуються умови для оптимального стиснення
        if (i + z_vec[i] == len(s)) and (len(s) % i == 0):
            # Якщо умови виконуються, повертаємо частину рядка s, яка буде стиснута
            return s[:i]
    # Якщо жодна пара (i, z_vec[i]) не відповідає умовам, повертаємо вихідний рядок s без змін
    return s

In [8]:
# Перевіримо роботу алгоритму стиснення рядка
s = "абырвалгабырвалгабырвалг"
print(z_func(s))
print(compress_with_z(s))

[24, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0]
абырвалг


## Відповіді на контрольні питання

### 1. Що таке «префікс-функція» у контексті алгоритмів на рядках, і як вона відрізняється від Z-функції?

**Префікс-функція** (або π-функція) для рядка s довжини n - це масив π довжини n, де π[i] - довжина найдовшого власного префікса підрядка s[0...i], який є одночасно суфіксом цього підрядка.

**Відмінності від Z-функції:**
1. Z-функція для позиції i показує довжину найдовшого спільного префікса між s[0...] та s[i...], тоді як префікс-функція показує довжину найдовшого спільного префікса і суфікса для підрядка s[0...i].
2. Префікс-функція використовується в алгоритмі Кнута-Морріса-Пратта для пошуку підрядка, а Z-функція має ширше застосування.
3. Префікс-функція розглядає кожен префікс окремо, а Z-функція порівнює весь рядок з його суфіксами.

### 2. Що таке Z-функція у вищому контексті алгоритмів на рядках, і яка її роль у вирішенні задач?

**Z-функція** для рядка s - це масив Z, де Z[i] - довжина найдовшого спільного префікса між s та суфіксом s, що починається з позиції i.

**Роль Z-функції у вирішенні задач:**
1. **Пошук підрядка в рядку**: Дозволяє ефективно знаходити всі входження шаблону в текст з лінійною складністю O(n+m).
2. **Стиснення рядка**: Допомагає знайти найкоротше представлення рядка у вигляді повторень деякого підрядка.
3. **Знаходження періодів рядка**: Дозволяє визначити, чи має рядок періодичну структуру.
4. **Знаходження найдовшого спільного префікса**: Допомагає знайти найдовший спільний префікс для множини рядків.
5. **Палліндромні підрядки**: Може бути використана для пошуку паліндромів у рядку.

Z-функція є потужним інструментом для аналізу рядків, який дозволяє вирішувати багато задач з лінійною складністю.

### 3. Які існують підходи до вирішення задачі «найдовший спільний підрядок» для двох рядків?

Для вирішення задачі знаходження найдовшого спільного підрядка (LCS - Longest Common Substring) для двох рядків існують такі підходи:

1. **Динамічне програмування**: Створюється таблиця DP[i][j], яка зберігає довжину найдовшого спільного підрядка, що закінчується в позиціях i та j відповідних рядків. Складність O(n*m).

2. **Суфіксні дерева**: Будується суфіксне дерево для конкатенації двох рядків з розділювачем. Найдовший спільний підрядок відповідає найглибшому вузлу, що має листя з обох рядків. Складність O(n+m).

3. **Суфіксні масиви**: Будується суфіксний масив для конкатенації рядків, потім шукаються сусідні суфікси з різних рядків і знаходиться їх найдовший спільний префікс. Складність O((n+m)log(n+m)).

4. **Алгоритм Рабіна-Карпа з бінарним пошуком**: Використовується бінарний пошук по довжині підрядка і хешування для перевірки наявності спільних підрядків заданої довжини. Складність O((n+m)log(min(n,m))).

5. **Z-функція**: Можна використовувати Z-функцію для знаходження спільних підрядків, обчислюючи її для конкатенації рядків з розділювачем.

### 4. Як можна застосувати алгоритми на рядках у задачах обробки природної мови або обробки текстів?

Алгоритми на рядках мають широке застосування в обробці природної мови (NLP) та обробці текстів:

1. **Пошук та індексація**: Алгоритми пошуку підрядків (KMP, Boyer-Moore, Z-функція) використовуються для швидкого пошуку ключових слів у великих текстах та створення пошукових індексів.

2. **Стемінг та лематизація**: Алгоритми для знаходження кореня слова або його нормальної форми часто використовують аналіз суфіксів та префіксів.

3. **Виявлення плагіату**: Алгоритми для знаходження найдовшого спільного підрядка або підпослідовності допомагають виявити скопійовані фрагменти тексту.

4. **Автоматичне виправлення помилок**: Алгоритми обчислення відстані редагування (Левенштейна, Дамерау-Левенштейна) використовуються для знаходження найближчих правильних слів.

5. **Стиснення тексту**: Алгоритми стиснення, що базуються на повторюваних патернах (LZ77, LZ78), використовують аналіз рядків для ефективного кодування.

6. **Аналіз n-грам**: Алгоритми для виділення та аналізу послідовностей слів використовуються для моделювання мови.

7. **Розпізнавання іменованих сутностей**: Алгоритми пошуку патернів допомагають виявляти імена, дати, місця тощо в тексті.

8. **Автоматичне реферування**: Алгоритми для знаходження ключових фраз та речень використовують аналіз частотності та структури тексту.

9. **Машинний переклад**: Алгоритми вирівнювання тексту допомагають знаходити відповідності між фразами різних мов.

## Висновки

У ході виконання лабораторної роботи було освоєно основні алгоритми на рядках засобами мови Python:

1. Реалізовано наївний алгоритм пошуку підрядка в рядку, який має асимптотичну складність O(n*m), де n - довжина тексту, m - довжина шаблону. Цей алгоритм простий у реалізації, але неефективний для великих текстів.

2. Реалізовано Z-функцію, яка обчислює для кожної позиції рядка довжину найдовшого спільного префікса, що починається з цієї позиції і з початку рядка. Z-функція має лінійну складність O(n).

3. На основі Z-функції реалізовано ефективний алгоритм пошуку підрядка в рядку з асимптотичною складністю O(n+m).

4. Реалізовано алгоритм стиснення рядка за допомогою Z-функції, який знаходить найкоротше представлення рядка у вигляді повторень деякого підрядка. Асимптотична складність цього алгоритму також O(n).

Таким чином, Z-функція є потужним інструментом для аналізу рядків, який дозволяє ефективно вирішувати різноманітні задачі обробки текстів з лінійною складністю.